<a href="https://colab.research.google.com/github/Devgoyal2808/facial_expression_recognition/blob/master/facial_expression_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/Devgoyal2808/facial_expressions

Cloning into 'facial_expressions'...
remote: Enumerating objects: 14214, done.
remote: Total 14214 (delta 0), reused 0 (delta 0), pack-reused 14214
Receiving objects: 100% (14214/14214), 239.65 MiB | 35.08 MiB/s, done.
Resolving deltas: 100% (223/223), done.
Checking out files: 100% (13996/13996), done.


In [5]:
import csv
data={}
with open('/content/facial_expressions/data/legend.csv') as f:
  reader=csv.reader(f)
  next(reader)              # to avoid col headers..
  for row in reader:
    key=row[2].lower()
    if key in data:
      data[key].append(row[1])
    else:
      data[key]=[row[1]]

In [6]:
emotion_list=list(data.keys())
emotion_list

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt']

In [11]:
import os

os.mkdir('master_data')
os.mkdir('master_data/training')
os.mkdir('master_data/testing')


In [12]:
for emotion in emotion_list:
  os.mkdir(os.path.join('master_data/training/',emotion))
  os.mkdir(os.path.join('master_data/testing/',emotion))

In [14]:
from shutil import copyfile
split_size=0.8

for emotion,images in data.items():
  train_size=int(split_size*len(images))
  train_images=images[:train_size]
  test_images=images[train_size:]
  for image in train_images:
    source=os.path.join('/content/facial_expressions/images',image)
    dest=os.path.join('/content/master_data/training',emotion,image)
    copyfile(source,dest)
  for image in test_images:
    source=os.path.join('/content/facial_expressions/images',image)
    dest=os.path.join('/content/master_data/testing',emotion,image)
    copyfile(source,dest)

In [16]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,Flatten

In [25]:
model=tf.keras.models.Sequential([
                                  Conv2D(16,[3,3],activation='relu',input_shape=(100,100,3)),
                                  MaxPooling2D(2,2),
                                  Conv2D(32,[3,3],activation='relu'),
                                  MaxPooling2D(2,2),
                                  Conv2D(64,[3,3],activation='relu'),
                                  MaxPooling2D(2,2),
                                  Flatten(),
                                  Dense(1024,activation='relu'),
                                  Dense(8,activation='softmax'),
])
model.compile(optimizer=Adam(lr=0.01),loss='categorical_crossentropy',metrics=['acc'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 98, 98, 16)        448       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 49, 49, 16)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 47, 47, 32)        4640      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 21, 21, 64)        18496     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 6400)             

In [26]:
train_dir='/content/master_data/training'
test_dir='/content/master_data/testing'

train_datagen=ImageDataGenerator(rescale=1.0/255)
train_generator=train_datagen.flow_from_directory(
    train_dir,
    target_size=(100,100),
    class_mode='categorical',
    batch_size=128
)

test_datagen=ImageDataGenerator(rescale=1.0/255)
test_generator=test_datagen.flow_from_directory(
    test_dir,
    target_size=(100,100),
    class_mode='categorical',
    batch_size=128
)


Found 10941 images belonging to 8 classes.
Found 2742 images belonging to 8 classes.


In [27]:
es=EarlyStopping(monitor='val_acc',patience=2,min_delta=0.01)         # if model is not improving atleast 1 per after 2 epochs stop it..

In [28]:
model.fit_generator(train_generator,
                    epochs=10,
                    verbose=1,
                    
                    validation_data=test_generator,
                    callbacks=[es])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
86/86 [==============================] - 105s 1s/step - loss: 3.3551 - acc: 0.4772 - val_loss: 1.0198 - val_acc: 0.4982
Epoch 2/10
86/86 [==============================] - 104s 1s/step - loss: 0.9330 - acc: 0.6195 - val_loss: 0.8925 - val_acc: 0.6805
Epoch 3/10
86/86 [==============================] - 105s 1s/step - loss: 0.7440 - acc: 0.7554 - val_loss: 0.9720 - val_acc: 0.6513
Epoch 4/10
86/86 [==============================] - 106s 1s/step - loss: 0.7016 - acc: 0.7620 - val_loss: 1.1860 - val_acc: 0.6273


In [29]:
# nearly 76% accuracy achieved...